In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

import os
if os.path.exists('cases.sql'):
    os.remove('cases.sql')

# Problem Recording

You might also want to attach a recorder to the `Problem` itself. This allows you to record an arbitrary case at a point of your choosing. This feature can be useful if you only record a limited number of variables during the run but would like to see a more complete list of values after the run.

The options are a subset of those for driver recording.

In [ ]:
import openmdao.api as om
om.show_options_table("openmdao.core.problem.Problem", recording_options=True)

```{note}
Note that the `excludes` option takes precedence over the `includes` option.
```

## Problem Recording Example

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src90", get_code("openmdao.test_suite.components.sellar.SellarDerivatives"), display=False)

:::{Admonition} `SellarDerivatives` class definition 
:class: dropdown

{glue:}`code_src90`
:::

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarDerivatives

import numpy as np

prob = om.Problem(model=SellarDerivatives())

model = prob.model
model.add_design_var('z', lower=np.array([-10.0, 0.0]),
                          upper=np.array([10.0, 10.0]))
model.add_design_var('x', lower=0.0, upper=10.0)
model.add_objective('obj')
model.add_constraint('con1', upper=0.0)
model.add_constraint('con2', upper=0.0)

prob.driver = om.ScipyOptimizeDriver(optimizer='SLSQP', tol=1e-9)

prob.add_recorder(om.SqliteRecorder("cases.sql"))

prob.recording_options['includes'] = []
prob.recording_options['record_objectives'] = True
prob.recording_options['record_constraints'] = True
prob.recording_options['record_desvars'] = True

prob.setup()
prob.run_driver()

In [ ]:
prob.record('final')
prob.cleanup()

cr = om.CaseReader("cases.sql")

# get list of cases recorded on problem
problem_cases = cr.list_cases('problem')

In [ ]:
assert problem_cases == ['final']

In [ ]:
# get list of output variables recorded on problem
cr.list_source_vars('problem')

In [ ]:
problem_vars = cr.list_source_vars('problem', out_stream=None)
assert sorted(problem_vars['outputs']) == ['con1', 'con2', 'obj', 'x', 'z']

In [ ]:
# get the recorded case
case = cr.get_case('final')

In [ ]:
case.get_objectives()

In [ ]:
assert_near_equal(case.get_objectives()['obj'], 3.18339395, 1e-8)

In [ ]:
case.get_design_vars()

In [ ]:
des_vars = case.get_design_vars()
assert_near_equal(des_vars['z'], [1.97763888, 5.87897722e-15], 1e-8)
assert_near_equal(des_vars['x'], 0., 1e-8)

In [ ]:
case.get_constraints()

In [ ]:
constraints = case.get_constraints()
assert_near_equal(constraints['con1'], 0, 1e-8)
assert_near_equal(constraints['con2'], -20.24472223, 1e-8)